In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.0'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA), 
                 ("train_standardize", imageStandardize), 
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_process_a", valPreprocessA), 
               ("val_standardize", imageStandardize), 
               ("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-11-09 21:37:48.796619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 21:37:49.289926: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4274 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:02:00.0, compute capability: 7.5


In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 227, 227, 3), dtype=float32, numpy=
array([[[[-1.4543149e+00, -1.5454352e+00, -1.6145902e+00],
         [-1.4543149e+00, -1.5332162e+00, -1.6145902e+00],
         [-1.4482555e+00, -1.5454352e+00, -1.6145902e+00],
         ...,
         [-1.4543149e+00, -1.5454352e+00, -1.6145902e+00],
         [-1.4543149e+00, -1.5454352e+00, -1.6145902e+00],
         [-1.4543149e+00, -1.5454352e+00, -1.6145902e+00]],

        [[-1.4543149e+00, -1.5454352e+00, -1.6145902e+00],
         [-1.4543149e+00, -1.5356700e+00, -1.6084807e+00],
         [-1.4543149e+00, -1.5326153e+00, -1.6145902e+00],
         ...,
         [-1.4543149e+00, -1.5454352e+00, -1.6145902e+00],
         [-1.4543149e+00, -1.5454352e+00, -1.6145902e+00],
         [-1.4543149e+00, -1.5454352e+00, -1.6145902e+00]],

        [[-1.4543149e+00, -1.5454352e+00, -1.6145902e+00],
         [-1.4543149e+00, -1.5448343e+00, -1.5901524e+00],
         [-1.4543149e+00, -1.5335668e+00, -1.6145902e+00],
         ...,
         [-

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)>

### Model

In [27]:
def createModel():
    inp = tf.keras.Input(shape=(CROP_HEIGHT, CROP_WIDTH, 3))

    backbone = tf.keras.applications.MobileNetV3Large(include_top=False, input_shape=(CROP_HEIGHT, CROP_WIDTH, 3))
    backbone.trainable = True
    
    finetune_layer_no = 10
    for layer in backbone.layers[:finetune_layer_no]:
        layer.trainable = False
    
    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = tf.keras.applications.mobilenet_v3.preprocess_input(inp)
    o = backbone(o, training=False)
    o = flat1(o)
    o = fc1(o)
    o = do1(o)

    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='baseline_1')
    
    return model

In [28]:
model = createModel()

In [29]:
model.summary()

Model: "baseline_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 MobilenetV3large (Functiona  (None, 8, 8, 960)        2996352   
 l)                                                              
                                                                 
 flat1 (Flatten)             (None, 61440)             0         
                                                                 
 fc1 (Dense)                 (None, 128)               7864448   
                                                                 
 do1 (Dropout)               (None, 128)               0         
                                                                 
 fc2 (Dense)                 (None, 128)               16512     
                                                        

### Compile Model

In [30]:
STEPS_PER_EPOCH = 1000
EPOCH = 100

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [31]:
def schedule(epoch, lr):
    return lr

In [33]:
logBasePath = "log/baseline_mobilenet_5"
logPrefix = "log"

In [34]:
for i, (train, val) in enumerate([folds[0]]):
    
    model = createModel()
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb, lrSchedule]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-11-09 21:38:25.489921: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-09 21:38:25.490061: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-09 21:38:25.490099: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-11-09 21:38:25.490547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/home/ntcadmin/env/trt-11.0/TensorRT-7.1.3.4/lib:
2022-11-09 21:38:25.490669: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/home/ntcadmin/env/trt-11.0/TensorRT-7.1.3.4/lib:
2022-11-09 21:38:25.490705: 

3528/3528 [==============================] - 47s 12ms/step - loss: 0.6937 - accuracy: 0.5232
Epoch 1/100
   5/1000 [..............................] - ETA: 36s - loss: 0.7014 - accuracy: 0.5000   

2022-11-09 21:39:17.220892: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-09 21:39:17.220941: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-09 21:39:17.221010: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2022-11-09 21:39:17.221042: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:184] cuptiSubscribe: ignored due to a previous error.
2022-11-09 21:39:17.221064: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:457] cuptiGetResultString: ignored due to a previous error.
2022-11-09 21:39:17.221085: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1716] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 


   9/1000 [..............................] - ETA: 36s - loss: 0.7344 - accuracy: 0.3889

2022-11-09 21:39:17.551670: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-09 21:39:17.552825: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:140] cuptiFinalize: ignored due to a previous error.
2022-11-09 21:39:17.552845: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:457] cuptiGetResultString: ignored due to a previous error.
2022-11-09 21:39:17.552852: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1808] function cupti_interface_->Finalize()failed with error 
2022-11-09 21:39:17.626994: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2022-11-09 21:39:17.627023: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2022-11-09 21:39:17.627029: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 0 callback api events and 0 a

...

2022-11-09 21:44:35.181479: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1887436800 exceeds 10% of free system memory.


1000/1000 [==============================] - 81s 81ms/step - loss: 0.5111 - accuracy: 0.7470 - val_loss: 0.5416 - val_accuracy: 0.7310 - lr: 1.0000e-05
Epoch 5/100
1000/1000 [==============================] - ETA: 0s - loss: 0.4767 - accuracy: 0.7763

2022-11-09 21:45:54.345522: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1887436800 exceeds 10% of free system memory.


1000/1000 [==============================] - 79s 79ms/step - loss: 0.4767 - accuracy: 0.7763 - val_loss: 0.5328 - val_accuracy: 0.7364 - lr: 1.0000e-05
Epoch 6/100
1000/1000 [==============================] - 77s 77ms/step - loss: 0.4542 - accuracy: 0.7908 - val_loss: 0.5050 - val_accuracy: 0.7563 - lr: 1.0000e-05
Epoch 7/100
1000/1000 [==============================] - 78s 78ms/step - loss: 0.5117 - accuracy: 0.7467 - val_loss: 0.4540 - val_accuracy: 0.7914 - lr: 1.0000e-05
Epoch 8/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.3981 - accuracy: 0.8202 - val_loss: 0.4497 - val_accuracy: 0.7863 - lr: 1.0000e-05
Epoch 9/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.4122 - accuracy: 0.8170 - val_loss: 0.4475 - val_accuracy: 0.7861 - lr: 1.0000e-05
Epoch 10/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.4241 - accuracy: 0.7985 - val_loss: 0.4295 - val_accuracy: 0.7936 - lr: 1.0000e-05
Epoch 11/100
1000/1000 [===

Epoch 55/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.1825 - accuracy: 0.9300 - val_loss: 0.3235 - val_accuracy: 0.8807 - lr: 1.0000e-05
Epoch 56/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.1860 - accuracy: 0.9245 - val_loss: 0.2279 - val_accuracy: 0.9046 - lr: 1.0000e-05
Epoch 57/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.1464 - accuracy: 0.9413 - val_loss: 0.2635 - val_accuracy: 0.9010 - lr: 1.0000e-05
Epoch 58/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.1642 - accuracy: 0.9373 - val_loss: 0.2936 - val_accuracy: 0.8749 - lr: 1.0000e-05
Epoch 59/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.1461 - accuracy: 0.9438 - val_loss: 0.2478 - val_accuracy: 0.9070 - lr: 1.0000e-05
Epoch 60/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.1927 - accuracy: 0.9195 - val_loss: 0.2622 - val_accuracy: 0.8974 - lr: 1.0000e-05
Epoch 61/1